In [23]:
import logging
import os
from pydicom import dcmread
from pynetdicom import AE, evt, debug_logger, StoragePresentationContexts
from pynetdicom.sop_class import Verification, CTImageStorage

dicom_file_path = "./dicom.dcm"

if os.path.exists(dicom_file_path):
    dataset = dcmread(dicom_file_path)
    
    # Check if the DICOM file is compressed and decompress if needed
    transfer_syntax = dataset.file_meta.TransferSyntaxUID
    is_compressed = False
    
    # Check for common compressed transfer syntaxes
    compressed_syntaxes = [
        '1.2.840.10008.1.2.4.50',  # JPEG Baseline
        '1.2.840.10008.1.2.4.51',  # JPEG Extended
        '1.2.840.10008.1.2.4.57',  # JPEG Lossless
        '1.2.840.10008.1.2.4.70',  # JPEG Lossless SV1
        '1.2.840.10008.1.2.4.80',  # JPEG-LS Lossless
        '1.2.840.10008.1.2.4.81',  # JPEG-LS Lossy
        '1.2.840.10008.1.2.4.90',  # JPEG 2000 Lossless
        '1.2.840.10008.1.2.4.91',  # JPEG 2000 Lossy
        '1.2.840.10008.1.2.5',     # RLE Lossless
    ]
    
    if transfer_syntax in compressed_syntaxes:
        is_compressed = True
        print(f"Decompressing DICOM with transfer syntax: {transfer_syntax}")
        dataset.decompress()
        print("DICOM decompressed successfully")
    
    ae = AE()
    ae.requested_contexts = StoragePresentationContexts[:127]
    association = ae.associate('127.0.0.1', 11112, ae_title="STORE_SCP")
    if association.is_established:
        print("Association established")
        status = association.send_c_store(dataset)
        # Check the status of the operation
        if status:
            status_code = getattr(status, "Status", None)
            if status_code == 0x0000:
                print("DICOM file sent successfully")
            else:
                print(f"Failed to send DICOM file. Status: {hex(status_code) if status_code else 'Unknown'}")
        association.release()
    else:
        print("Association rejected, testing ABORT")
        association.abort()
else:
    print(f"DICOM file not found: {dicom_file_path}")

I: Requesting Association
D: Request Parameters:
D: ======================= OUTGOING A-ASSOCIATE-RQ PDU ========================
D: Our Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.1.0
D: Our Implementation Version Name:   PYNETDICOM_210
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    PYNETDICOM
D: Called Application Name:     STORE_SCP
D: Our Max PDU Receive Size:    16382
D: Presentation Contexts:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =Computed Radiography Image Storage
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D:   Context ID:        3 (Proposed)
D:     Abstract Syntax: =Encapsulated PDF Storage


D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D:   Context ID:        5 (Proposed)
D:     Abstract Syntax: =Encapsulated CDA Storage
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D:   Context ID:        7 (Proposed)
D:     Abstract Syntax: =Encapsulated STL Storage
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D:   Context ID:        9 (Proposed)
D:     Abstract Syntax: =Encapsulated OBJ Storage
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxe

Association rejected, testing ABORT


In [24]:
from pynetdicom import AE
from pynetdicom.sop_class import Verification

ae = AE()

ae.add_requested_context(Verification)

association = ae.associate('127.0.0.1', 11112, ae_title="STORE_SCP")
if association.is_established:
    print("Association established")
    association.release()
else:
    print("Association rejected, testing ABORT")
    association.abort()


I: Requesting Association
D: Request Parameters:
D: ======================= OUTGOING A-ASSOCIATE-RQ PDU ========================
D: Our Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.1.0
D: Our Implementation Version Name:   PYNETDICOM_210
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    PYNETDICOM
D: Called Application Name:     STORE_SCP
D: Our Max PDU Receive Size:    16382
D: Presentation Context:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =Verification SOP Class
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D: Requested Extended Negotiation: None
D: Requested Common Extended Negotiation: None
D: Requested Asynchronous Operations Window Negotiation: None
D: Requested User Identity Negotiation: None
D: ========================== END A-ASSOCIA

Association established


In [9]:
from pynetdicom import AE, debug_logger
from pydicom.dataset import Dataset
from pynetdicom.sop_class import ModalityWorklistInformationFind

debug_logger()
ae = AE()
ae.add_requested_context(ModalityWorklistInformationFind)
ds = Dataset()
ds.PatientName = '*'
ds.ScheduledProcedureStepSequence = [Dataset()]
item = ds.ScheduledProcedureStepSequence[0]
item.ScheduledStationAETitle = 'CTSCANNER'
item.ScheduledProcedureStepStartDate = '20181005'
item.Modality = 'CT'

association = ae.associate('127.0.0.1', 11112)

if association.is_established:
    print("Association established")
    responses = association.send_c_find(ds, ModalityWorklistInformationFind)
    for (status, identifier) in responses:
        if status:
            print(f"C-FIND : {format(status.Status)}")
        else:
            print(f"C-FIND request for {identifier} failed")
    association.release()
else:
    print("Association rejected, testing ABORT")
    association.abort()


I: Requesting Association
D: Request Parameters:
D: ======================= OUTGOING A-ASSOCIATE-RQ PDU ========================
D: Our Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.1.0
D: Our Implementation Version Name:   PYNETDICOM_210
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    PYNETDICOM
D: Called Application Name:     ANY-SCP
D: Our Max PDU Receive Size:    16382
D: Presentation Context:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =Modality Worklist Information Model - FIND
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D: Requested Extended Negotiation: None
D: Requested Common Extended Negotiation: None
D: Requested Asynchronous Operations Window Negotiation: None
D: Requested User Identity Negotiation: None
D: ======================

Association rejected, testing ABORT
